# Importanto Libs

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, Dropdown
import plotly.express as px
from dash import Dash, dcc, html, Output, Input

# Lendo dataset

In [25]:
df_merged = pd.read_csv('../data/processed/dataset_merged.csv')
df_merged

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,review_answer_timestamp,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,month,year,year_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,2017-10-12 03:43:48,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,10,2017,2017-10
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,2017-10-12 03:43:48,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,10,2017,2017-10
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,2017-10-12 03:43:48,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,10,2017,2017-10
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,2018-08-08 18:37:50,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,7,2018,2018-07
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,2018-08-22 19:07:58,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,8,2018,2018-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117324,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,...,2018-03-02 17:50:01,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10,2,2018,2018-02
117325,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,...,2017-09-22 23:10:57,1,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02,8,2017,2017-08
117326,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,...,2018-01-27 09:16:56,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59,1,2018,2018-01
117327,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,...,2018-01-27 09:16:56,2,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59,1,2018,2018-01


In [26]:
merged = df_merged.copy()

# Gráficos

## Número de pedidos por mês

In [27]:
col_estado = "customer_state"

# Agregar pedidos por estado, ano e ano_mes
pedidos = (
    merged.groupby([col_estado, "year", "year_month"], as_index=False)["order_id"]
    .nunique()
    .rename(columns={"order_id": "num_orders"})
)
# listas para os dropdowns
estados = sorted(pedidos[col_estado].dropna().unique())
anos = sorted(pedidos["year"].dropna().unique())

dropdown_estado = widgets.Dropdown(
    options=estados,
    value=estados[0],
    description="Estado:",
    disabled=False
)

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="Ano:",
    disabled=False
)

output = widgets.Output()

def atualizar_grafico(change=None):
    with output:
        output.clear_output()
        
        estado = dropdown_estado.value
        ano = dropdown_ano.value
        
        filtrado = pedidos[
            (pedidos[col_estado] == estado) &
            (pedidos["year"] == ano)
        ].sort_values("year_month")
        
        if filtrado.empty:
            print(f"Sem dados para {estado} em {ano}")
            return
        
        fig = px.line(
            filtrado,
            x="year_month",
            y="num_orders",
            markers=True,
            title=f"Pedidos por mês em {estado} no ano {ano}",
            labels={
                "year_month": "Ano-mês",
                "num_orders": "Número de pedidos"
            }
        )
        fig.update_layout(xaxis_tickangle=-45)
        fig.show()

# conectar os widgets à função
dropdown_estado.observe(atualizar_grafico, names="value")
dropdown_ano.observe(atualizar_grafico, names="value")

# mostrar os widgets e o gráfico
display(widgets.HBox([dropdown_estado, dropdown_ano]))
display(output)

# desenhar o gráfico inicial
atualizar_grafico()


Output()

## Receita total por estado

In [28]:
merged["order_purchase_timestamp"] = pd.to_datetime(merged["order_purchase_timestamp"])
merged["year"] = merged["order_purchase_timestamp"].dt.year

col_estado = "customer_state"


In [29]:
receita = (
   merged
   .groupby([col_estado, "year"], as_index=False)["payment_value"]
   .sum()
   .rename(columns={"payment_value": "total_receita"})
)


In [ ]:
anos = sorted(receita["year"].unique())

dropdown_ano = widgets.Dropdown(
   options=anos,
   value=anos[0],
   description="Ano:",
   disabled=False
)


output = widgets.Output()

def atualizar_grafico(change=None):
   with output:
      output.clear_output()
      
      ano = dropdown_ano.value
      
      filtrado = (
         receita[receita["year"] == ano]
         .sort_values("total_receita", ascending=False)
      )
      
      fig = px.bar(
         filtrado,
         x=col_estado,
         y="total_receita",
         title=f"Receita total por estado em {ano}",
         labels={
               col_estado: "Estado",
               "total_receita": "Receita Total (R$)",
         },
         color="customer_state",
         height=1200,
         width=2000,
      )
      
      fig.update_layout(xaxis_tickangle=-45)
      fig.show()

dropdown_ano.observe(atualizar_grafico, names="value")

display(dropdown_ano)
display(output)

# gerar gráfico inicial
atualizar_grafico()